# # RTSMADlib Demo

## Simple Credit Application Model and deployment

In this sample we demonstrate a credit application approval probability model using Random Forest. The application demonstrates the feature generation and model invocation realtime. so the payload from the client applications is passed thru a chain. The feature engine will take the raw input and creates feature set for model and then model is used to make a prediction. The model is deployed as workflow. There will be an orchestrator component, a feature engine, a model. The orchestrator component is reponsible to run the flow.

### Pre-Requisites
1. A running instance of Greenplum with MADlib
2. A running instance of kubernetes environment
3. Modify database connection parameters whereever we see place holders

### The notebook perform the below tasks

1. Connect to greenplum and setup the session
2. Create the schema and create all the tables needed, Load test the data to Greenpulm
3. Build and train model
4. Test Batch Score the model .
5. Operationalize model with RTSMADlib
6. Test the ML flow Service
7. Undeploy ML flow application containers.

# connect to Greenplum

In [ ]:
import psycopg2               # Python-PostgreSQL Database Adapter - https://pypi.python.org/pypi/psycopg2
import pandas as pd           # Python Data Analysis Library - https://pandas.pydata.org/
import math  
import json

%load_ext sql

# PLEASE MODIFY THE BELOW AS PER YOUR GREENPLUM CLUSTER SETTINGS
database_host = '35.196.46.152'
database_databasename = 'gpadmin'
database_username = 'gpadmin'
database_password = 'qg7lGPyhxEQmj'
database_port = '5432'

try:
    connString = "host='{}' dbname='{}' user='{}' password='{}' port={}".format(database_host,database_databasename,database_username,database_password,database_port)
    # print connString
    conn = psycopg2.connect(connString)
    cur = conn.cursor()
    conn.autocommit = True
        
    connection = 'postgresql://{}:{}@{}:{}/{}'.format(database_username,database_password,database_host,database_port,database_databasename)
    %sql $connection

    message = "<span style='color:green'>**Connection successful!**</span>"
    print(message)
except Exception as e:
    message = "<span style='color:red'>**ERROR: Unable to connect to the database ({})**</span>".format(e)
    print(message) 

# Schema Creation 

In [ ]:
%%sql

/*
Credit Application Approval Model
jvawdrey@pivotal.io
*/


DO $$
BEGIN

    IF NOT EXISTS(
        SELECT schema_name
          FROM information_schema.schemata
          WHERE schema_name = 'credit'
      )
    THEN
      EXECUTE 'CREATE SCHEMA credit';
    END IF;

END
$$;


 -- Create table via external source;
DROP EXTERNAL TABLE IF EXISTS credit.credit_application_external;
CREATE EXTERNAL WEB TABLE credit.credit_application_external (
    a1 varchar(1)
   ,a2 float
   ,a3 float
   ,a4 varchar(1)
   ,a5 varchar(2)
   ,a6 varchar(2)
   ,a7 varchar(2)
   ,a8 float
   ,a9 boolean
   ,a10 boolean
   ,a11 float
   ,a12 boolean
   ,a13 varchar(1)
   ,a14 float
   ,a15 float
   ,a16 varchar(1)
) LOCATION ('http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data')
FORMAT 'CSV'
(NULL AS '?');

# Impute in missing values

In [ ]:
%%sql
DROP TABLE IF EXISTS credit.credit_application_data;
CREATE TABLE credit.credit_application_data AS
SELECT row_number() OVER() AS _id
      ,coalesce(a1,'b') AS a1
      ,coalesce(a2, avg(a2) OVER()) AS a2
      ,coalesce(a3, avg(a3) OVER()) AS a3
      ,coalesce(a4, 'u') AS a4
      ,coalesce(a5, 'g') AS a5
      ,coalesce(a6, 'c') AS a6
      ,coalesce(a7, 'v') AS a7
      ,coalesce(a8, avg(a8) OVER()) AS a8
      ,coalesce(a9, True) AS a9
      ,coalesce(a10, False) AS a10
      ,coalesce(a11, 0) AS a11
      ,coalesce(a12, False) AS a12
      ,coalesce(a13, 'g') AS a13
      ,coalesce(a14, avg(a14) OVER()) AS a14
      ,coalesce(a15, avg(a15) OVER()) AS a15
      ,CASE WHEN a16 = '+' THEN 1 ELSE 0 END AS a16
FROM credit.credit_application_external
DISTRIBUTED RANDOMLY;


#   Continous model inputs

In [ ]:
%%sql
DROP TABLE IF EXISTS credit.model_inputs_cont;
CREATE TABLE credit.model_inputs_cont AS
SELECT _id
      ,a16 AS approval
      ,a2
      ,a3
      ,a8
      ,a11
      ,a14
      ,a15
FROM credit.credit_application_data
DISTRIBUTED BY (_id);

SELECT * FROM credit.model_inputs_cont LIMIT 10;

#  Categorical model inputs

In [ ]:
%%sql
DROP TABLE IF EXISTS credit.model_inputs_cat;
SELECT madlib.encode_categorical_variables (
    'credit.credit_application_data',
    'credit.model_inputs_cat',
    'a1,a4,a5,a6,a7,a9,a10,a12,a13',
    NULL,
    '_id',
    NULL,
    'a1=b, a4=y, a5=p, a6=x, a7=z, a9=false, a10=false, a12=false, a13=s'
);

select * from credit.model_inputs_cat;

# Combine model tables

In [ ]:
%%sql 
DROP TABLE IF EXISTS credit.model_inputs;
CREATE TABLE credit.model_inputs AS
SELECT *
FROM credit.model_inputs_cat
JOIN credit.model_inputs_cont
USING (_id);

select * from credit.model_inputs;

# Split traing and test data.

In [ ]:
%%sql
DROP TABLE IF EXISTS credit.model
                    ,credit.model_train
                    ,credit.model_test;
SELECT madlib.train_test_split(
    'credit.model_inputs',
    'credit.model',
    0.7,
    NULL,
    NULL,
    '*',
    FALSE,
    TRUE
);


# Train random forest model

In [ ]:
%%sql

DROP TABLE IF EXISTS credit.rf_model, credit.rf_model_summary, credit.rf_model_group;
SELECT madlib.forest_train(
        'credit.model_train',
        'credit.rf_model',
        '_id',
        'approval',
        'a2,a3,a8,a11,a14,a15,a1_a,a4_l,a4_u,a5_g,a5_gg,a6_aa,a6_c,a6_cc,a6_d,a6_e,a6_ff,a6_i,a6_j,a6_k,a6_m,a6_q,a6_r,a6_w,a7_bb,a7_dd,a7_ff,a7_h,a7_j,a7_n,a7_o,a7_v,a9_true,a10_true,a12_true,a13_g,a13_p',
        null,
        null,
        10::integer,
        5::integer,
        true::boolean,
        5::integer,
        10::integer,
        3::integer,
        1::integer,
        10::integer
    );

# Score test data

In [ ]:
%%sql

DROP TABLE IF EXISTS credit.model_test_scored;
SELECT madlib.forest_predict('credit.rf_model',
                             'credit.model_test',
                             'credit.model_test_scored',
                             'prob');

-- Combine scores with original table
DROP TABLE IF EXISTS credit.model_test_scored_tmp;
CREATE TABLE credit.model_test_scored_tmp AS
SELECT *
FROM credit.model_test_scored
JOIN credit.model_test
USING (_id);
DROP TABLE credit.model_test_scored;
ALTER TABLE credit.model_test_scored_tmp RENAME TO model_test_scored;


-- Calculate AUC
DROP TABLE IF EXISTS credit.model_test_scored_auc;
SELECT madlib.area_under_roc(
    'credit.model_test_scored'
   ,'credit.model_test_scored_auc'
   ,'estimated_prob_1'
   ,'approval'
);
SELECT *
FROM credit.model_test_scored_auc;

# Feature Engineering

In [ ]:
%%sql
DROP TABLE IF EXISTS credit.model_train;
CREATE TABLE credit.model_train AS
SELECT 1 AS _id,a2,a3,a8,a11,a14,a15,
       CASE WHEN a1 = 'a' THEN 1 ELSE 0 END AS a1_a, 
       CASE WHEN a4 = 'l' THEN 1 ELSE 0 END AS a4_l, 
       CASE WHEN a4 = 'u' THEN 1 ELSE 0 END AS a4_u, 
       CASE WHEN a5 = 'g' THEN 1 ELSE 0 END AS a5_g, 
       CASE WHEN a5 = 'gg' THEN 1 ELSE 0 END AS a5_gg, 
       CASE WHEN a6 = 'aa' THEN 1 ELSE 0 END AS a6_aa, 
       CASE WHEN a6 = 'cc' THEN 1 ELSE 0 END AS a6_cc, 
       CASE WHEN a6 = 'd' THEN 1 ELSE 0 END AS a6_d, 
       CASE WHEN a6 = 'ff' THEN 1 ELSE 0 END AS a6_ff, 
       CASE WHEN a6 = 'j' THEN 1 ELSE 0 END AS a6_j, 
       CASE WHEN a6 = 'r' THEN 1 ELSE 0 END AS a6_r, 
       CASE WHEN a6 = 'c' THEN 1 ELSE 0 END AS a6_c, 
       CASE WHEN a6 = 'e' THEN 1 ELSE 0 END AS a6_e, 
       CASE WHEN a6 = 'i' THEN 1 ELSE 0 END AS a6_i, 
       CASE WHEN a6 = 'k' THEN 1 ELSE 0 END AS a6_k, 
       CASE WHEN a6 = 'm' THEN 1 ELSE 0 END AS a6_m, 
       CASE WHEN a6 = 'q' THEN 1 ELSE 0 END AS a6_q, 
       CASE WHEN a6 = 'w' THEN 1 ELSE 0 END AS a6_w, 
       CASE WHEN a7 = 'bb' THEN 1 ELSE 0 END AS a7_bb, 
       CASE WHEN a7 = 'dd' THEN 1 ELSE 0 END AS a7_dd, 
       CASE WHEN a7 = 'ff' THEN 1 ELSE 0 END AS a7_ff, 
       CASE WHEN a7 = 'h' THEN 1 ELSE 0 END AS a7_h, 
       CASE WHEN a7 = 'j' THEN 1 ELSE 0 END AS a7_j, 
       CASE WHEN a7 = 'n' THEN 1 ELSE 0 END AS a7_n, 
       CASE WHEN a7 = 'v' THEN 1 ELSE 0 END AS a7_v, 
       CASE WHEN a7 = 'o' THEN 1 ELSE 0 END AS a7_o, 
       CASE WHEN a9 = 'True' THEN 1 ELSE 0 END AS a9_True, 
       CASE WHEN a10 = 'True' THEN 1 ELSE 0 END AS a10_True, 
       CASE WHEN a12 = 'True' THEN 1 ELSE 0 END AS a12_True, 
       CASE WHEN a13 = 'p' THEN 1 ELSE 0 END AS a13_p, 
       CASE WHEN a13 = 'g' THEN 1 ELSE 0 END AS a13_g

FROM (
  SELECT coalesce(a1,'b') AS a1
        ,coalesce(a2, avg(a2) OVER()) AS a2
        ,coalesce(a3, avg(a3) OVER()) AS a3
        ,coalesce(a4, 'u') AS a4
        ,coalesce(a5, 'g') AS a5
        ,coalesce(a6, 'c') AS a6
        ,coalesce(a7, 'v') AS a7
        ,coalesce(a8, avg(a8) OVER()) AS a8
        ,coalesce(a9, True) AS a9
        ,coalesce(a10, False) AS a10
        ,coalesce(a11, 0) AS a11
        ,coalesce(a12, False) AS a12
        ,coalesce(a13, 'g') AS a13
        ,coalesce(a14, avg(a14) OVER()) AS a14
        ,coalesce(a15, avg(a15) OVER()) AS a15
        ,CASE WHEN a16 = '+' THEN 1 ELSE 0 END AS a16
FROM (
    -- Change to select from 'message table'
    -- For example selecting random record from original data for scoring
    SELECT *
    FROM credit.credit_application_external
    ORDER BY random()
  ) foo
) bar
distributed RANDOMLY;

# Prediction

In [ ]:
%%sql
DROP TABLE IF EXISTS credit.scored_results;

SELECT madlib.forest_predict('credit.rf_model','credit.model_inputs','credit.scored_results','prob');

SELECT estimated_prob_1, estimated_prob_0 FROM credit.scored_results limit 10;


# RTSMADlib
Operationalize the model
The MADlib model from Greenplum is containerized and deployed container management system. In this case we are using local docker environment. The rtsmadlib tool will take care of how to bundle, deploy and serve the model as REST endpoint.

In [62]:
!rts4madlib --help

usage: rts4madlib [-h] [--name NAME] [--action {deploy,undeploy}]
                  [--type {flow,model,feature-engine,featurecache,batch}]
                  [--target {docker,kubernetes}] [--inputJson [INPUTJSON]]

optional arguments:
  -h, --help            show this help message and exit
  --name NAME           unique name for module
  --action {deploy,undeploy}
  --type {flow,model,feature-engine,featurecache,batch}
  --target {docker,kubernetes}
  --inputJson [INPUTJSON]
                        input for corresponding module


# Deployment Manifest

In [71]:
#  deployment specification for madlib model
model=  {
"modeldb-datasource.jdbc-url": "jdbc:postgresql://35.196.46.152:5432/gpadmin",
		"modeldb-datasource.userName": "gpadmin",
		"modeldb-datasource.password": "qg7lGPyhxEQmj",
		"madlibrest.modelname": "Credit Approval Random Forest model",
		"madlibrest.modeldescription": "Credit Approval Random Forest Classification Example",
		"madlibrest.modelschema": "credit",
		"madlibrest.modeltables": [
			"rf_model",
			"rf_model_group",
			"rf_model_summary"
		],
		"madlibrest.modelinputtable": "model_inputs",
		"madlibrest.modelquery": "SELECT madlib.forest_predict('credit.rf_model','credit.model_inputs','credit.scored_results','prob')",
		"madlibrest.resultstable": "scored_results",
		"madlibrest.resultsquery": "SELECT estimated_prob_1, estimated_prob_0 FROM credit.scored_results;"
	}

In [72]:
#  deployment specification for madlib feature engine

featuresengine =  {
  "spring.profiles.active": "nocache",
  "modeldb-datasource": {
    "jdbc-url": "jdbc:postgresql://35.196.46.152:5432/gpadmin",
    "userName": "gpadmin",
    "password": "qg7lGPyhxEQmj"
  },
  "feature-engine": {
    "featurename": "CreditApplicationFeatures",
    "featuredescription": "Credit Application Feature engine",
    "featuresschema": "credit",
    "payloadtable": "message",
    "featurequery": "SELECT 1 AS _id,a2,a3,a8,a11,a14,a15, CASE WHEN a1 = 'a' THEN 1 ELSE 0 END AS a1_a, CASE WHEN a4 = 'l' THEN 1 ELSE 0 END AS a4_l, CASE WHEN a4 = 'u' THEN 1 ELSE 0 END AS a4_u, CASE WHEN a5 = 'g' THEN 1 ELSE 0 END AS a5_g, CASE WHEN a5 = 'gg' THEN 1 ELSE 0 END AS a5_gg, CASE WHEN a6 = 'aa' THEN 1 ELSE 0 END AS a6_aa, CASE WHEN a6 = 'cc' THEN 1 ELSE 0 END AS a6_cc, CASE WHEN a6 = 'd' THEN 1 ELSE 0 END AS a6_d, CASE WHEN a6 = 'ff' THEN 1 ELSE 0 END AS a6_ff, CASE WHEN a6 = 'j' THEN 1 ELSE 0 END AS a6_j, CASE WHEN a6 = 'r' THEN 1 ELSE 0 END AS a6_r, CASE WHEN a6 = 'c' THEN 1 ELSE 0 END AS a6_c, CASE WHEN a6 = 'e' THEN 1 ELSE 0 END AS a6_e, CASE WHEN a6 = 'i' THEN 1 ELSE 0 END AS a6_i, CASE WHEN a6 = 'k' THEN 1 ELSE 0 END AS a6_k, CASE WHEN a6 = 'm' THEN 1 ELSE 0 END AS a6_m, CASE WHEN a6 = 'q' THEN 1 ELSE 0 END AS a6_q, CASE WHEN a6 = 'w' THEN 1 ELSE 0 END AS a6_w, CASE WHEN a7 = 'bb' THEN 1 ELSE 0 END AS a7_bb, CASE WHEN a7 = 'dd' THEN 1 ELSE 0 END AS a7_dd, CASE WHEN a7 = 'ff' THEN 1 ELSE 0 END AS a7_ff, CASE WHEN a7 = 'h' THEN 1 ELSE 0 END AS a7_h, CASE WHEN a7 = 'j' THEN 1 ELSE 0 END AS a7_j, CASE WHEN a7 = 'n' THEN 1 ELSE 0 END AS a7_n, CASE WHEN a7 = 'v' THEN 1 ELSE 0 END AS a7_v, CASE WHEN a7 = 'o' THEN 1 ELSE 0 END AS a7_o, CASE WHEN a9 = 'True' THEN 1 ELSE 0 END AS a9_True, CASE WHEN a10 = 'True' THEN 1 ELSE 0 END AS a10_True, CASE WHEN a12 = 'True' THEN 1 ELSE 0 END AS a12_True, CASE WHEN a13 = 'p' THEN 1 ELSE 0 END AS a13_p, CASE WHEN a13 = 'g' THEN 1 ELSE 0 END AS a13_g FROM ( SELECT coalesce(a1,'b') AS a1 ,coalesce(a2, avg(a2) OVER()) AS a2 ,coalesce(a3, avg(a3) OVER()) AS a3 ,coalesce(a4, 'u') AS a4 ,coalesce(a5, 'g') AS a5 ,coalesce(a6, 'c') AS a6 ,coalesce(a7, 'v') AS a7 ,coalesce(a8, avg(a8) OVER()) AS a8 ,coalesce(a9, 'True') AS a9 ,coalesce(a10, 'False') AS a10 ,coalesce(a11, 0) AS a11 ,coalesce(a12, 'False') AS a12 ,coalesce(a13, 'g') AS a13 ,coalesce(a14, avg(a14) OVER()) AS a14 ,coalesce(a15, avg(a15) OVER()) AS a15 ,CASE WHEN a16 = '+' THEN 1 ELSE 0 END AS a16 FROM ( SELECT * FROM credit.message ) t1 ) t2",
    "cacheenabled": "false",
    "cacheentities": {
      "key": "val1"
    }
  }
}

In [73]:
myconfig=json.dumps ({
	"deploy-packages": [
		"model",
		"feature-engine"
	],
	"model": model,
	"feature-engine": featuresengine
  }
)


with open("config.json", "w") as f:
    f.write(myconfig)
    

In [77]:
! echo "Deploying Credit Model to kubernetes......"
! source ~/.bash_profile
! rts4madlib --name credit-app-v1 --type flow --target kubernetes --action deploy --inputJson  config.json

Deploying Credit Model to kubernetes......
deployName => credit-app-v1


processing deployment package =>model
Deploying credit-app-v1-rts-for-madlib-model container to kubernetes ..........
deleting any old service configurations.....
Error from server (NotFound): configmaps "credit-app-v1-rts-for-madlib-model-config" not found
No existing configuration found. Continuing....
creating new service configuration.....
configmap/credit-app-v1-rts-for-madlib-model-config created
creating the deployment .....
deployment.apps/credit-app-v1-rts-for-madlib-model created
service/credit-app-v1-rts-for-madlib-model created
Polling for pod <credit-app-v1-rts-for-madlib-model>'s running status ........
Status => ContainerCreating

Status => Running
POD NAME => credit-app-v1-rts-for-madlib-model-66d5866bdb-rvl57

Provisioning is finished.


processing deployment package =>feature-engine
Deploying credit-app-v1-rts-for-madlib-featuresengine container to kubernetes ..........
deleting any old service c

In [83]:
! kubectl get all

NAME                                                               READY     STATUS    RESTARTS   AGE
pod/credit-app-v1-rts-for-madlib-featuresengine-5f797865b7-9d5sq   1/1       Running   0          2m12s
pod/credit-app-v1-rts-for-madlib-mlmodelflow-5d6c4bc85-qpdch       1/1       Running   0          2m6s
pod/credit-app-v1-rts-for-madlib-model-66d5866bdb-rvl57            1/1       Running   0          2m18s

NAME                                                  TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)                         AGE
service/credit-app-v1-rts-for-madlib-featuresengine   LoadBalancer   10.36.6.208    35.227.55.124    8086:32525/TCP                  2m11s
service/credit-app-v1-rts-for-madlib-mlmodelflow      LoadBalancer   10.36.13.118   35.243.129.230   8089:30677/TCP                  2m6s
service/credit-app-v1-rts-for-madlib-model            LoadBalancer   10.36.10.225   34.73.240.170    8085:30362/TCP,5432:32618/TCP   2m18s
service/kubernetes               

In [81]:
# Model container
! curl -v -H "Content-Type:application/json" http://34.73.240.170:8085/actuator/info

*   Trying 34.73.240.170...
* TCP_NODELAY set
* Connected to 34.73.240.170 (34.73.240.170) port 8085 (#0)
> GET /actuator/info HTTP/1.1
> Host: 34.73.240.170:8085
> User-Agent: curl/7.63.0
> Accept: */*
> Content-Type:application/json
> 
< HTTP/1.1 200 
< Content-Type: application/vnd.spring-boot.actuator.v2+json;charset=UTF-8
< Transfer-Encoding: chunked
< Date: Tue, 01 Oct 2019 04:49:21 GMT
< 
* Connection #0 to host 34.73.240.170 left intact
{"MADlib Model - Name":"Credit Approval Random Forest model","Description":"Credit Approval Random Forest Classification Example","Model Table(s)":["rf_model","rf_model_group","rf_model_summary"],"Actor Table":"model_inputs","Results Table":"scored_results","Results Query":"SELECT estimated_prob_1, estimated_prob_0 FROM credit.scored_results;"}

In [82]:
# feature engine container
! curl -v -H "Content-Type:application/json" http://35.227.55.124:8086/actuator/info

*   Trying 35.227.55.124...
* TCP_NODELAY set
* Connected to 35.227.55.124 (35.227.55.124) port 8086 (#0)
> GET /actuator/info HTTP/1.1
> Host: 35.227.55.124:8086
> User-Agent: curl/7.63.0
> Accept: */*
> Content-Type:application/json
> 
< HTTP/1.1 200 
< Content-Type: application/vnd.spring-boot.actuator.v2+json;charset=UTF-8
< Transfer-Encoding: chunked
< Date: Tue, 01 Oct 2019 04:49:30 GMT
< 
* Connection #0 to host 35.227.55.124 left intact
{}

In [84]:
! curl -v -H "Content-Type:application/json" http://35.243.129.230:8089/predict -d '{"a1":"a","a2":58.67,"a3":4.46,"a4":"u","a5":"g","a6":"q","a7":"h","a8":3.04,"a9":"t","a10":"t","a11":6.0,"a12":"f","a13":"g","a14":43.0,"a15":560.0,"a16":"+"}'

*   Trying 35.243.129.230...
* TCP_NODELAY set
* Connected to 35.243.129.230 (35.243.129.230) port 8089 (#0)
> POST /predict HTTP/1.1
> Host: 35.243.129.230:8089
> User-Agent: curl/7.63.0
> Accept: */*
> Content-Type:application/json
> Content-Length: 159
> 
* upload completely sent off: 159 out of 159 bytes
< HTTP/1.1 200 
< Content-Type: application/json;charset=UTF-8
< Transfer-Encoding: chunked
< Date: Tue, 01 Oct 2019 04:50:02 GMT
< 
* Connection #0 to host 35.243.129.230 left intact
[{"estimated_prob_1":1.0,"estimated_prob_0":0.0}]

In [85]:
! echo "Undeploying Credit Application Model......"
! source ~/.bash_profile
! rts4madlib --name credit-app-v1 --type flow --target kubernetes --action undeploy

Undeploying Credit Application Model......
deployName => credit-app-v1
undeploying credit-app-v1-rts-for-madlib-model .........
deployment.extensions "credit-app-v1-rts-for-madlib-model" deleted
service "credit-app-v1-rts-for-madlib-model" deleted
configmap "credit-app-v1-rts-for-madlib-model-config" deleted
undeploying credit-app-v1-rts-for-madlib-featuresengine .........
deployment.extensions "credit-app-v1-rts-for-madlib-featuresengine" deleted
service "credit-app-v1-rts-for-madlib-featuresengine" deleted
configmap "credit-app-v1-rts-for-madlib-featuresengine-config" deleted
undeploying credit-app-v1-rts-for-madlib-mlmodelflow .........
deployment.extensions "credit-app-v1-rts-for-madlib-mlmodelflow" deleted
service "credit-app-v1-rts-for-madlib-mlmodelflow" deleted
configmap "credit-app-v1-rts-for-madlib-mlmodelflow-config" deleted


In [87]:
! kubectl get all

NAME                                                               READY     STATUS        RESTARTS   AGE
pod/credit-app-v1-rts-for-madlib-featuresengine-5f797865b7-9d5sq   1/1       Terminating   0          3m19s
pod/credit-app-v1-rts-for-madlib-model-66d5866bdb-rvl57            1/1       Terminating   0          3m25s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.36.0.1    <none>        443/TCP   3h34m
